## Processing TnSeq mutant trajectory data for classification

The transposon sequencing data contains counts for insertion mutants over the course of the fitness assay. Here, I will process this data into a set of features that can be used as an input for the machine learning classification algorithms.

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from Bio.SeqIO.FastaIO import SimpleFastaParser
import re
import pandas as pd
import seaborn as sns
import os
from pathlib import Path
import itertools

In [ ]:
#current working directory
repo = os.getcwd()
print(repo)

In [ ]:
# sns.set_style('ticks')
sns.set_theme()
sns.set_context('paper')

### Loading all the metadata for this project

In [ ]:
metadata_path = repo +'/Metadata/'
data_path = repo + '/Data'

In [ ]:
#opening the pandas file with all the metadata!
all_data = pd.read_csv(metadata_path+"all_metadata_REL606.txt", sep="\t")
names = all_data.iloc[:,0]
gene_start = all_data.iloc[:,3]
gene_end = all_data.iloc[:,4]
strand = all_data.iloc[:,5]
locations = np.transpose(np.vstack([gene_start,gene_end,strand]))
k12_tags = all_data.iloc[:,2]
uniprot_rel606 = all_data.iloc[:,6]
product = all_data.iloc[:,-1]

In [ ]:
#fractions of the gene at the 5' and 3' ends to be excluded from analysis because they insertions there may not actually
#be disruptive to protein function
frac5p = 0.1
frac3p = 0.25

with open(metadata_path+"rel606_reference.fasta") as in_handle:
    for title, seq in SimpleFastaParser(in_handle):
        ta_sites = [m.start(0) for m in re.finditer('TA', seq)]
ta_sites = np.array(ta_sites)

#counting how many TA sites are present in each gene
ta_gene = np.zeros(len(names))
for i in range(0,len(names)):
    start = locations[i, 0]
    end = locations[i, 1]
    length = end - start
    #if the gene is on the forward strand
    if locations[i,2]==1:
        #counting sites only in the middle 80% of the gene, excluding 10% at each end
        ta_gene[i] = np.sum((ta_sites > start+length*frac5p)&(ta_sites < end - length*frac3p))
    elif locations[i,2]==-1:
        ta_gene[i] = np.sum((ta_sites < start+length*frac5p)&(ta_sites > end - length*frac3p))

### Compiling all the data for the sequencing experiment

Organization of counts_data:
- column 0: TA site coordinate
- column 1: reads at t0, no UMI correction
- column 2: reads at t0, UMI correction

We are interested in column 2: the data at t0 will allow us to classify genes as essential or not in LB (as the selection occurred in LB)

In [ ]:
counts_data = np.loadtxt(data_path+'/green_methods_new_merged_all_TAsites.txt')

In [ ]:
def downsample(data, n_downsampled, random_seed):
    """
    Inputs: 
    - data: counts matrix for bulk fitness assay
    - scale: scaling factor for downsampling, must be greater than 1.
    
    Process:
    - downsample number of reads mapping to an insertion site as follows (for each time point, here: by row)
    - use np.repeat to get an list with every insertion site repeated N times, where N is the number of mapped reads
    - use np.shuffle to rearrange this list
    - pick the first 1/scale fraction of this list
    - use np.unique to which sites are represented, and how frequently after downsampling.
    
    Output:
    - data_scaled: same shape as data but each row of the matrix downsampled by the scaling factor
    """
    
    assert data.sum() >= n_downsampled, f"n_downsampled must be less than sum(data)"
    
    if data.sum() == n_downsampled: #do not downsample the data at all:
        return data
    
    else:
        np.random.seed(random_seed)
        data_scaled = np.zeros_like(data)
        #this is the key step in the process, every TA site is repeated as many times as number of reads mapping to it
        explicit_data = np.repeat(np.arange(0,data.shape[0]), data.astype('int'))
        #this list is then shuffled
        np.random.shuffle(explicit_data)
        #as we shuffled the data, taking the first N_ds reads is equivalent to taking a 1/scale random subset of the data
        downsampled = explicit_data[:n_downsampled]
        #getting the counts and unique TA sites represented after downsampling
        unique, counts = np.unique(downsampled, return_counts=True)
        data_scaled[unique] = counts
        
        return data_scaled

In [ ]:
norm = 10**7  # we are normalizing our data to this value

In [ ]:
data_ds = downsample(counts_data[2],n_downsampled=norm, random_seed=42) #

### Some more useful functions for extracting specific regions of a gene

In [ ]:
def search_gene_interior(locations,ta_sites,i):
    start = locations[i, 0]
    end = locations[i, 1]
    length = end - start
    #if the gene is on the forward strand
    if locations[i,2]==1:
        search_area = (ta_sites > start+length*frac5p)&(ta_sites < end - length*frac3p)
    #if the gene is on the reverse strand
    elif locations[i,2]==-1:
        search_area = (ta_sites < start+length*frac5p)&(ta_sites > end - length*frac3p)
    return search_area

In [ ]:
def search_gene_5p(locations,ta_sites,i):
    start = locations[i, 0]
    end = locations[i, 1]
    length = end - start
    #if the gene is on the forward strand
    if locations[i,2]==1:
        search_area = (ta_sites < start+length*frac5p)&(ta_sites > start)
    #if the gene is on the reverse strand
    elif locations[i,2]==-1:
        search_area = (ta_sites > start+length*frac5p)&(ta_sites < start)

    return search_area

In [ ]:
def search_gene_3p(locations,ta_sites,i):
    start = locations[i, 0]
    end = locations[i, 1]
    length = end - start
    #if the gene is on the forward strand
    if locations[i,2]==1:
        search_area = (ta_sites > end-length*frac3p)&(ta_sites < end)
    #if the gene is on the reverse strand
    elif locations[i,2]==-1:
        search_area = (ta_sites < end-length*frac3p)&(ta_sites > end)
    return search_area

## Brainstorming set of features to extract for each gene

- ta_sites: number of TA insertion sites within the gene interior
- length: length of gene
- reads5p: number of reads mapping to TA sites at the 5' end of the gene
- reads3p: number of reads mapping to TA sites at the 3' end of the gene
- normalized_coverage: number of reads mapping to the interior of the gene

All the metrics calculated henceforth are using an the normalized coverage

- insertion_index: normalized_coverage/length of gene interior (gene length*0.65)
- f_zeros: fraction of TA sites with zero reads mapped
- f_min_thresh: fraction of TA sites with at least min_thresh reads
- zero_interval: extent of longest string of 0 coverage sites 
- median: median number of reads mapping to the gene interior
- upper25: interquartile range Q3
- lower25: interquartile range Q1

As a first pass, I will normalize coverage to ~10 million reads

In [ ]:
rows_list = []

In [ ]:
def construct_rows(gene, thresh):
    """
    Input: gene number
    Output: a dictionary containing values for all the features for gene
    """
    
    if ta_gene[gene] < 1: # if there are no sites in the interior of the gene (which can happen sometimes)
        
        return None
    
    else:
        row_dict = {}
        row_dict['Gene'] = gene
        row_dict['Gene_name'] = names[gene]
        row_dict['TA_sites_interior'] = ta_gene[gene]
        row_dict['Gene_length'] = np.abs(locations[gene,1]-locations[gene,0])
        if np.sum(search_gene_5p(locations,ta_sites,gene))==0:
            row_dict['Mean_counts_5p_10pct'] = 0
        else:
            row_dict['Mean_counts_5p_10pct'] = np.mean(data_ds[search_gene_5p(locations,ta_sites,gene)])
        
        if np.sum(search_gene_3p(locations,ta_sites,gene))==0:
            row_dict['Mean_counts_3p_25pct'] = 0
        else:
            row_dict['Mean_counts_3p_25pct'] = np.mean(data_ds[search_gene_3p(locations,ta_sites,gene)])
        gene_interior = data_ds[search_gene_interior(locations,ta_sites,gene)]
        row_dict['Mean_counts_interior'] = np.mean(gene_interior)
        row_dict['Insertion_index'] = np.mean(gene_interior)/np.abs(locations[gene,1]-locations[gene,0])/0.65
        row_dict['Fraction_zeros'] = np.sum(gene_interior==0)/ta_gene[gene]
        row_dict['Fraction_above_thresh'] = np.sum(gene_interior>thresh)/ta_gene[gene]
        row_dict['Median_counts'] = np.median(gene_interior)
        row_dict['Upper25'] = np.percentile(gene_interior,75)
        row_dict['Lower25'] = np.percentile(gene_interior,25)
        if np.min(gene_interior)==0:
            max_zeros = max(len(list(y)) for (c,y) in itertools.groupby(gene_interior) if c==0)
        else:
            max_zeros = 0
        row_dict['Zeros_interval'] = max_zeros/ta_gene[gene]
        
        return row_dict

In [ ]:
thresh = 5 #parameter that can be tuned (ideally should scale with norm)
#rule of thumb: thresh = norm/(2*10**6)

In [ ]:
rows_list = [construct_rows(gene,thresh=thresh) for gene in range(len(names))]

In [ ]:
rows_list = list(filter(None, rows_list))

In [ ]:
processed_data = pd.DataFrame(rows_list)

In [ ]:
processed_data = processed_data.set_index('Gene', drop=True)

In [ ]:
processed_data.to_csv('tnseq_features_REL606.csv')